In [10]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [11]:
from collections import namedtuple
ARGS_FOR_LIST = namedtuple('FOR_LIST', ('dir', 'ratio', 'seed', 'csv'))

def get_for_list():
    dir_list = ['p10000', 'p20000', 'p30000']
    ratio_list = ['r4', 'r5', 'r6']
    seed_list = ['s'+str(123 + i) for i in range(1)]
    csv_list = ['census', 'mobile']

    return ARGS_FOR_LIST(dir_list, ratio_list, seed_list, csv_list)

In [12]:
def get_read_path(name):
    path = '/Users/kessapassa/OneDrive/research_log/20190331/' + name + '/'
    return path

def get_file_name(args):
    return args.dir + args.ratio + args.seed + '_' + args.csv + '.csv'

def get_full_path(name, args):
    return get_read_path(name) + get_file_name(args)

In [13]:
def for_default_init(func, array):
    for_list = get_for_list()
    
    for _dir in for_list.dir:
        array[_dir] = {}
        
        for _ratio in for_list.ratio:
            array[_dir][_ratio] = {}
            
            for _seed in for_list.seed:
                array[_dir][_ratio][_seed] = {}
                
                for _csv in for_list.csv:
                    array[_dir][_ratio][_seed][_csv] = {}
                    args = ARGS_FOR_LIST(_dir, _ratio, _seed, _csv)
                    func(args, array)

In [15]:
csv_array = {}
def main(args, array):
    df = pd.read_csv(get_full_path('Origin', args), 
                     encoding='Shift_JISx0213')
    df = df.loc[:, ['id', 'time', 'area']]

    group_list = df.groupby(['id'], sort=True)
    for _id, _group in group_list:
        _group = _group.sort_values(['time'])
        array[args.dir][args.ratio][args.seed][args.csv][_id] = []
        for area in np.asanyarray(_group.loc[:, 'area']):
            array[args.dir][args.ratio][args.seed][args.csv][_id].append(area)
            
for_default_init(main, csv_array)

In [17]:
od_array = {}
def create_od(args, array):
    df = pd.DataFrame()
    df_main = csv_array[args.dir][args.ratio][args.seed][args.csv].copy()
    for _id in df_main:
        df_main[_id].insert(0, _id)
        tmp = pd.DataFrame(df_main[_id]).T
        df = pd.concat([df, tmp])
        
#     df.to_csv(get_full_path('od_from_bigdata', args), index=False)
    array[args.dir][args.ratio][args.seed][args.csv] = df
    
    
for_default_init(create_od, od_array)